Google API YT v3 documentation

https://developers.google.com/youtube/v3/docs/

In [1]:
import json
import pprint
import time
import sqlite3
from datetime import datetime

import more_itertools as miter
import requests
import urllib.request
from bs4 import BeautifulSoup

import utility as util
import utility.i
import utility.l

import google.oauth2.credentials
import google_auth_oauthlib.flow
from googleapiclient.discovery import build
from googleapiclient.errors import HttpError
from google_auth_oauthlib.flow import InstalledAppFlow

GOOGLE_SECRETS = "../.secrets/google.json"
with open(GOOGLE_SECRETS) as f:
    GOOGLE_API_KEY = json.load(f)["key"]
CLIENT_SECRETS_FILE = '../.secrets/client_secret.json'
SCOPES = ['https://www.googleapis.com/auth/youtube']
API_HOST_NAME = "www.googleapis.com"
API_SERVICE_NAME = "youtube"
API_VERSION = 'v3'
    
pp = pprint.PrettyPrinter(indent=4)

In [2]:
def query_to_str(query):
    o = []
    for k, v in query.items():
        if v is None:
            continue
        if isinstance(v, list):
            v = ",".join(v)
        else:
            v = str(v)
        k = str(k)
        o.append(k + "=" + v)
    return "&".join(o)

def create_tables_if_not_exists(conn):
    conn.execute("""
    CREATE TABLE IF NOT EXISTS Channel(
        id text PRIMARY KEY,
        title text,
        description blob,
        n_videos integer,
        view_count integer,
        subscriber_count integer,
        upload_playlist_id text
    )""")
    conn.execute("""
    CREATE TABLE IF NOT EXISTS Topic(
        id text PRIMARY KEY,
        category text
    )""")
    conn.execute("""
    CREATE TABLE IF NOT EXISTS ChannelTopic(
        channel_id text,
        topic_id text,
        FOREIGN KEY(channel_id) REFERENCES Channel(id),
        FOREIGN KEY(topic_id)   REFERENCES Topic(id),
        UNIQUE(channel_id, topic_id)
    )
    """)
    conn.commit()

def drop_tables_if_not_exists(conn):
    conn.execute("""
    DROP TABLE IF EXISTS Channel
    """)
    conn.execute("""
    DROP TABLE IF EXISTS Topic
    """)
    conn.execute("""
    DROP TABLE IF EXISTS ChannelTopic
    """)
    conn.commit()
    
def get_subscriptions():
    """Do OAuth 2.0 authenticated session to get my YT private data."""
    flow = InstalledAppFlow.from_client_secrets_file(CLIENT_SECRETS_FILE, SCOPES)
    credentials = flow.run_console()
    with build(API_SERVICE_NAME, API_VERSION, credentials=credentials) as youtube:
        # Get all the subscriptions
        print("Running: ", end=" ")
        pageToken = None; count = 0; items = []
        while True:
            payload = youtube.subscriptions().list(
                part="id,contentDetails,snippet",
                maxResults=50,
                mine=True,
                pageToken=pageToken,
            ).execute()
            items.extend(payload["items"])
            count += 1
            print("o ", end=" ")
            try:
                pageToken = payload["nextPageToken"]
            except KeyError:
                break
        print()
        print(f"Made {count} requests")
        print(f"Got {len(items)} subscriptions")
    return items

def get_channel_details():
    with sqlite3.connect('youtube.db') as conn:
        cur = conn.execute(f"SELECT id FROM Channel")
        res = util.i.chain.from_iterable(cur)
        items = []; count = 0
        for chunk in util.i.chunked(res, 20):
            # Make HTTP GET request
            query = {
                "key": GOOGLE_API_KEY,
                "id": chunk,
                "part": [
                    "contentDetails",
                    "id",
                    "snippet",
                    "statistics",
                    "status",
                    "topicDetails",
                ]
            }
            query = query_to_str(query)
            resource = "channels"
            url = f"https://{API_HOST_NAME}/{API_SERVICE_NAME}/{API_VERSION}/{resource}?{query}"
            try:
                res = requests.get(url)
                payload = json.loads(res.content)
                items.extend(payload["items"])
                count += 1
                print("o ", end=" ")
            except Exception as e:
                print("Exception:", e)
        print()
        print(f"Made {count} requests")
        print(f"Got {len(items)} channels")
    return items

In [33]:
# to reset, drop all tables
with sqlite3.connect('youtube.db') as conn:
    drop_tables_if_not_exists(conn)

In [34]:
# save all subscribed channels from subscriptions.
items = get_subscriptions()
with sqlite3.connect('youtube.db') as conn:
    create_tables_if_not_exists(conn)
    _input = []
    for item in items:
        snippet = item["snippet"]
        channelId = snippet["resourceId"]["channelId"]
        title = snippet["title"]
        _input.append((channelId, title))
    conn.executemany("""
    INSERT OR IGNORE INTO
    Channel(id, title)
    VALUES (?, ?)
    """, _input)
    conn.commit()
    del _input

Please visit this URL to authorize this application: https://accounts.google.com/o/oauth2/auth?response_type=code&client_id=517970623738-ctnfm8dhn55jbmrtsn1kmb4i68isl2ah.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fyoutube&state=2CUdkrKeRcj1W9kcByA2HaAIFPmwb1&prompt=consent&access_type=offline
Enter the authorization code: 4/1AX4XfWg2sRDPPeK9LNK-qUzp_obiLEZDNkDmJUAIIv0QtK1i7ZPm9QI8j40
Running:  o  o  o  o  o  o  o  o  o  o  o  
Made 11 requests
Got 532 subscriptions


In [3]:
# sanity check the subscriptions
with sqlite3.connect('youtube.db') as conn:
    _key = "title"
    cur = conn.execute(f"SELECT {_key} FROM Channel")
    res = cur.fetchall()
    print("Number of channels:", len(res))
    for i, row in zip(range(10), res):
        print(i, row[0])
    print("...")

Number of channels: 532
0 Andreas Kling
1 Adiyogi
2 World Armwrestling League
3 Michelle Khare
4 The Armchair Historian
5 Lyle Forever
6 albie
7 Max0r
8 Simulation
9 Speak The Truth
...


In [12]:
# sanity check for table creation
with sqlite3.connect('youtube.db') as conn:
    # query tables created
    res = conn.execute("SELECT * FROM sqlite_master WHERE type='table';")
    while True:
        try:
            for desc, value in zip(res.description, res.fetchone()):
                print(desc[0], ": ", value)
        except TypeError:
            break

type :  table
name :  Channel
tbl_name :  Channel
rootpage :  2
sql :  CREATE TABLE Channel(
        id text PRIMARY KEY,
        title text,
        description blob,
        n_videos integer,
        view_count integer,
        subscriber_count integer,
        upload_playlist_id text
    )
type :  table
name :  Topic
tbl_name :  Topic
rootpage :  4
sql :  CREATE TABLE Topic(
        id text PRIMARY KEY,
        detail text
    )
type :  table
name :  ChannelTopic
tbl_name :  ChannelTopic
rootpage :  6
sql :  CREATE TABLE ChannelTopic(
        channel_id text,
        topic_id text,
        FOREIGN KEY(channel_id) REFERENCES Channel(id),
        FOREIGN KEY(topic_id)   REFERENCES Topic(id),
        UNIQUE(channel_id, topic_id)
    )


In [5]:
# Extracting channels and topic labels
items = get_channel_details()
with sqlite3.connect('youtube.db') as conn:
    _input = []
    for item in items:
        snippet = item["snippet"]
        description = snippet["description"]
        statistics = item["statistics"]
        viewCount = statistics["viewCount"]
        hiddenSubscriberCount = statistics["hiddenSubscriberCount"]
        if hiddenSubscriberCount:
            subscriberCount = 0
        else:
            subscriberCount = statistics["subscriberCount"]
        videoCount = statistics["videoCount"]
        uploadPlaylist = item["contentDetails"]["relatedPlaylists"]["uploads"]
        channel_id = item["id"]
        _input.append((
            description,
            viewCount,
            subscriberCount,
            videoCount,
            uploadPlaylist,
            channel_id
        ))
    conn.executemany("""
    UPDATE Channel
    SET description = ?,
        view_count = ?,
        subscriber_count = ?,
        n_videos = ?,
        upload_playlist_id = ?
    WHERE id = ?
    """, _input)
    conn.commit()
    del _input
    
    _input_1 = {}
    _input_2 = []
    for item in items:
        channel_id = item["id"]
        if "topicDetails" in item and len(item["topicDetails"]) > 0:
            topicDetails = item["topicDetails"]
            for topicId, topicCategory in zip(
                topicDetails["topicIds"], topicDetails["topicCategories"]
            ):
                _input_1[topicId] = topicCategory
                _input_2.append((channel_id, topicId))
        else:
            snippet = item["snippet"]
            title = snippet["title"]
            print(f"{title} ({channel_id}) doesn't have topics, skipping...")
    _input_1 = list(_input_1.items())
    conn.executemany("""
    INSERT OR IGNORE INTO Topic VALUES (?, ?)
    """, _input_1)
    conn.executemany("""
    INSERT OR IGNORE INTO ChannelTopic VALUES (?, ?)
    """, _input_2)
    conn.commit()
    del _input_1
    del _input_2

o  o  o  o  o  o  o  o  o  o  o  o  o  o  o  o  o  o  o  o  o  o  o  o  o  o  o  
Made 27 requests
Got 532 channels
Veritasium (UC-ImLFXGIe2FC4Wo5hOodnw) doesn't have topics, skipping...
Anca Dragan (UC4QZ--BLZn0wmUXq8g60gxw) doesn't have topics, skipping...
Anthony van der Meer (UCD8QpDKSsn6xzGWDMispV7g) doesn't have topics, skipping...
Kadi Runnels (UCL1hIPwFVgRr_PXeM3LmUsQ) doesn't have topics, skipping...
Martin Shkreli (UCe_TgByANr-hShag3_17sNw) doesn't have topics, skipping...
MLRG UBC (UCuDrZai1J71y4L9dP-xQ44A) doesn't have topics, skipping...
Isaac Krementsov (UCur-jsGydCfJe1rdWzMGl_A) doesn't have topics, skipping...
Carolyn Shen (UCxe7fxguk5j5r6Vfmx4gQgA) doesn't have topics, skipping...


In [31]:
# sanity check the channels
with sqlite3.connect('youtube.db') as conn:
    _key = "id, title, n_videos, view_count, subscriber_count"
    cur = conn.execute(f"SELECT {_key} FROM Channel")
    res = cur.fetchall()
    for i, row in zip(range(10), res):
        channel_id, title, n_videos, view_count, subscriber_count = row
        print(*row)
        _cur = conn.execute("SELECT topic_id FROM ChannelTopic WHERE channel_id=?", [channel_id])
        _res = _cur.fetchall()
        print("    ", *util.i.chain.from_iterable(_res))
        
        

    print("...")
    

UC3ts8coMP645hZw9JSD3pqQ Andreas Kling 885 2728530 33500
     /m/019_rr /m/01k8wb /m/07c1v
UCsXV4t9YfUeDuZ276hE20Jg Adiyogi 353 43685543 375000
     /m/019_rr /m/06bvp /m/098wr
UC1pdbN6j_txoNhXVWVtouuQ World Armwrestling League 353 132014927 257000
     /m/019_rr /m/06ntj
UCGGZ_POGmIWG1pQXTDzQv-g Michelle Khare 189 263163762 2760000
     /m/019_rr /m/027x7n
UCeUJFQ0D9qs6aVNyUt9fkeQ The Armchair Historian 140 205833525 1670000
     /m/01k8wb /m/025zzc /m/02ntfj /m/03hf_rm /m/0bzvm2
UCMbvTF91O--bhwmg9dTMAPA Lyle Forever 244 4661322 110000
     /m/019_rr /m/0bzvm2
UCrDQW9kAElm707c5z6d5r7Q albie 62 61794158 529000
     /m/028sqc /m/04rlf /m/064t9 /m/0ggq0m
UCfgh3Ul_dG6plQ7rzuOLx-w Max0r 81 62691329 832000
     /m/025zzc /m/02ntfj /m/03hf_rm /m/0403l3g /m/0bzvm2
UC6JhS4GvWf3AJfOTfkrse2w Simulation 1490 20963354 87300
     /m/019_rr /m/01k8wb
UCvQ9wJKXJwcw_4tNaHHbWdA Speak The Truth 296 53769620 279000
     /m/098wr
...


In [27]:
? util.i.

In [6]:
def get_channel_details():
    """Get details for each YT channel saved in DB."""
    with sqlite3.connect('youtube.db') as conn:
        cur = conn.execute(f"SELECT id FROM Channel")
        res = util.i.chain.from_iterable(cur)
        items = []; count = 0
        for chunk in util.i.chunked(res, 20):
            # Make HTTP GET request
            query = {
                "key": GOOGLE_API_KEY,
                "id": chunk,
                "part": [
                    "contentDetails",
                    "id",
                    "snippet",
                    "statistics",
                    "status",
                    "topicDetails",
                ]
            }
            query = query_to_str(query)
            resource = "channels"
            url = f"https://{API_HOST_NAME}/{API_SERVICE_NAME}/{API_VERSION}/{resource}?{query}"
            try:
                res = requests.get(url)
                payload = json.loads(res.content)
                items.extend(payload["items"])
                count += 1
                print("o ", end=" ")
            except Exception as e:
                print("Exception:", e)
        print()
        print(f"Made {count} requests")
        print(f"Got {len(items)} channels")
    return items

def get_uploaded_videos_from_playlist(playlist_id):
    """Given a playlist ID, get all the videos from the playlist."""
    pageToken = None
    items = []; count = 0
    while True:
        query = {
            "key": GOOGLE_API_KEY,
            "playlistId": playlist_id,
            "maxResults": 50,
            "part": "snippet",
            "pageToken": pageToken
        }
        query = query_to_str(query)
        resource = "playlistItems"
        url = f"https://{API_HOST_NAME}/{API_SERVICE_NAME}/{API_VERSION}/{resource}?{query}"
        try:
            res = requests.get(url)
            payload = json.loads(res.content)
            items.extend(payload["items"])
            count += 1
            print("o ", end=" ")
        except Exception as e:
            print("Exception:", e)
    print()
    print(f"Made {count} requests")
    print(f"Got {len(items)} channels")
    return items

def get_video_details():
    with sqlite3.connect('youtube.db') as conn:
        cur = conn.execute(f"SELECT id FROM Video")
        res = util.i.chain.from_iterable(cur)
        items = []; count = 0
        for chunk in util.i.chunked(res, 20):
            pass

In [ ]:
# Get all videos from the (upload) playlist
pageToken = None
count = 0
items = []
while True:
    host = "www.googleapis.com"
    path = "youtube/v3/playlistItems"
    query = {
        "key": GOOGLE_API_KEY,
        "playlistId": "UURLQIvQcoF41xZdrtKMXSvg",
        "maxResults": 50,
        "part": "snippet",
        "pageToken": pageToken
    }
    query = query_to_str(query)
    query = f"https://{host}/{path}?{query}"
    try:
        res = requests.get(query)
        payload = json.loads(res.content)
        items.extend(payload["items"])
        count += 1
        print("o ", end=" ")
        try:
            pageToken = payload["nextPageToken"]
        except KeyError:
            break    
    except Exception as e:
        print("Exception:", e)

print()
print(f"Made {count} requests")
print(f"Got {len(items)} videos")

In [ ]:
# List all the videos we go earlier, and print the links to the videos
should_print_link = True
for item in items:
    snippet = item["snippet"]
    videoId = snippet["resourceId"]["videoId"]
    title = snippet["title"]
    description = snippet["description"]
    description = description.replace("\n"," ")[:80]
    if should_print_link:
        print(f"https://youtu.be/{videoId}", end=" ")
        print(f", title: [ {title} ]")
    else:
        print(f"videoId: {videoId} , title: [ {title} ]")
    print("    ", description)
    print()

## Misc.

In [28]:
# Get details about a video given video ID
# Based on:
# https://developers.google.com/youtube/v3/getting-started
# See:
# https://stackoverflow.com/questions/48857015/how-do-i-add-an-api-key-to-this-youtube-data-api-request
host = "www.googleapis.com"
path = "youtube/v3/videos"
query = {
    "key": GOOGLE_API_KEY,
    "id": "kO9HmhwGzXs",
    "part": "snippet,contentDetails,statistics,status",
}
query = query_to_str(query)
query = f"https://{host}/{path}?{query}"
try:
    res = requests.get(query)
    print("Status code:", res.status_code)
    payload = json.loads(res.content)
except Exception as e:
    print("Exception:", e)

print()
print("===Content===")
item = payload["items"][0]
videoId = item["id"]
snippet = item["snippet"]
title = snippet["title"]
description = snippet["description"]
channelTitle = snippet["channelTitle"]
tags = snippet["tags"]

print("video ID:", videoId)
print("title:", title)
print("description:")
print("[[[", description.replace("\n", " ")[:200], "]]]")
print("channel title:", channelTitle)
print("tags:", *tags)
print("statistics:")
for k, v in item["statistics"].items():
    print("    ", k, v)

Status code: 200

===Content===
video ID: kO9HmhwGzXs
title: An Incorrect Summary of Elden Ring | Part 1
description:
[[[ Part 1: The Moon and Stars ━━━━━━━━━━━━━━━━━━━━━━━━━━ Follow me on Twitter! https://twitter.com/realMax0r Patreon: https://www.patreon.com/max0r Join the Max0r Discord! https://discord.gg/max0r Twitch ]]]
channel title: Max0r
tags: maxor max0r elden ring maxor elden ring maxor ring maxor dark souls max0r dark souls elden ring sseth SsethTzeentach Uberdanger Martincitopants
statistics:
     viewCount 1881815
     likeCount 142163
     favoriteCount 0
     commentCount 11043


In [29]:
# Get infomation about channel given channel ID
# See:
# https://developers.google.com/youtube/v3/docs/channels
host = "www.googleapis.com"
path = "youtube/v3/channels"
query = {
    "key": GOOGLE_API_KEY,
    "id": "UCRLQIvQcoF41xZdrtKMXSvg",
    "part": [
#         "auditDetails", # 403 - The caller does not have permission
        "brandingSettings",
        "contentDetails",
        "contentOwnerDetails",
        "id",
        "localizations",
        "snippet",
        "statistics",
        "status",
        "topicDetails",
    ]
}
query = query_to_str(query)
query = f"https://{host}/{path}?{query}"
try:
    res = requests.get(query)
    print("Status code:", res.status_code)
    payload = json.loads(res.content)
except Exception as e:
    print("Exception:", e)

print()
print("===Content===")
item = payload["items"][0]
print("Statistics:")
for k, v in item["statistics"].items():
    print("    ", k, v)
relatedPlaylists = item["contentDetails"]["relatedPlaylists"]
print("relatedPlaylists:")
for k, v in relatedPlaylists.items():
    print("    ", k, v)
snippet = item["snippet"]
title = snippet["title"]
description = snippet["description"]
print("title:", title)
print("description:")
print(description.replace("\n", " ")[:200])

Status code: 200

===Content===
Statistics:
     viewCount 7879865
     subscriberCount 83500
     hiddenSubscriberCount False
     videoCount 313
relatedPlaylists:
     likes 
     uploads UURLQIvQcoF41xZdrtKMXSvg
title: Lectures Beyond Beyond
description:
Welcome! This channel is an archive/library of U.G. Krishnamurti recordings, videos and interviews.


In [149]:
# Get videos from the (upload) playlist
# We can't query using id (this is playlist item ID). Must use playlistId
# We got the uploads playlist ID by calling channels.list
# A playlist can be accessed from the YT website via
# https://www.youtube.com/playlist?list=UURLQIvQcoF41xZdrtKMXSvg
host = "www.googleapis.com"
path = "youtube/v3/playlistItems"
query = {
    "key": GOOGLE_API_KEY,
    "playlistId": "UURLQIvQcoF41xZdrtKMXSvg",
    "maxResults": 20,
    "part": [
        "id", "contentDetails", "snippet", "status"
    ]
}
query = query_to_str(query)
query = f"https://{host}/{path}?{query}"
try:
    res = requests.get(query)
    print("Status code:", res.status_code)
    payload = json.loads(res.content)
except Exception as e:
    print("Exception:", e)

print()
print("===Content===")
print("pageInfo:")
for k, v in payload["pageInfo"].items():
    print("    ", k, v)
print("prevPageToken:", payload.get("prevPageToken", None))
print("nextPageToken:", payload.get("nextPageToken", None))
    
# each playlist item result (in items) is one of
# https://developers.google.com/resources/api-libraries/documentation/youtube/v3/csharp/latest/classGoogle_1_1Apis_1_1YouTube_1_1v3_1_1Data_1_1PlaylistItem.html
print()
print("===Item 0===")
item = payload["items"][0]
playlistItemId = item["id"]
privacyStatus = item["status"]["privacyStatus"]
print("playlistItemId:", playlistItemId)
print("privacyStatus:", privacyStatus)
contentDetails = item["contentDetails"]
videoId = contentDetails["videoId"]
videoPublishedAt = datetime.strptime(contentDetails["videoPublishedAt"], "%Y-%m-%dT%H:%M:%SZ")
print("videoId:", videoId)
print("videoPublishedAt:", videoPublishedAt)
snippet = item["snippet"]
title = snippet["title"]
description = snippet["description"].replace("\n", " ")[:200]
print("title:", title)
print("description:", description)
# redundant information
videoId = snippet["resourceId"]["videoId"]
videoPublishedAt = datetime.strptime(snippet["publishedAt"], "%Y-%m-%dT%H:%M:%SZ")
channelTitle = snippet["videoOwnerChannelTitle"]
channelId = snippet["videoOwnerChannelId"]
print("From channel", channelTitle, ", channelId:", channelId)

Status code: 200

===Content===
pageInfo:
     totalResults 313
     resultsPerPage 20
prevPageToken: None
nextPageToken: EAAaBlBUOkNCUQ

===Item 0===
playlistItemId: VVVSTFFJdlFjb0Y0MXhaZHJ0S01YU3ZnLnlXcVg4d281YWxR
privacyStatus: public
videoId: yWqX8wo5alQ
videoPublishedAt: 2022-04-01 19:40:18
title: Woman Left Child, Husband, Family Because of U.G. She Says. Hear His Reply...
description: "I'm not responsible."  Books on U.G. Krishnamurti: Mind is a Myth - https://amzn.to/3erGeYi The Courage to Stand Alone: Conversations with U.G. Krishnamurti - https://amzn.to/3enKIiU This Dog Barking
From channel Lectures Beyond Beyond , channelId: UCRLQIvQcoF41xZdrtKMXSvg


In [154]:
# Get all videos from the (upload) playlist
pageToken = None
count = 0
items = []
while True:
    host = "www.googleapis.com"
    path = "youtube/v3/playlistItems"
    query = {
        "key": GOOGLE_API_KEY,
        "playlistId": "UURLQIvQcoF41xZdrtKMXSvg",
        "maxResults": 50,
        "part": "snippet",
        "pageToken": pageToken
    }
    query = query_to_str(query)
    query = f"https://{host}/{path}?{query}"
    try:
        res = requests.get(query)
        payload = json.loads(res.content)
        items.extend(payload["items"])
        count += 1
        print("o ", end=" ")
        try:
            pageToken = payload["nextPageToken"]
        except KeyError:
            break    
    except Exception as e:
        print("Exception:", e)

print()
print(f"Made {count} requests")
print(f"Got {len(items)} videos")

o  o  o  o  o  o  o  
Made 7 requests
Got 313 videos


In [155]:
# List all the videos we go earlier, and print the links to the videos
should_print_link = True
for item in items:
    snippet = item["snippet"]
    videoId = snippet["resourceId"]["videoId"]
    title = snippet["title"]
    description = snippet["description"]
    description = description.replace("\n"," ")[:80]
    if should_print_link:
        print(f"https://youtu.be/{videoId}", end=" ")
        print(f", title: [ {title} ]")
    else:
        print(f"videoId: {videoId} , title: [ {title} ]")
    print("    ", description)
    print()

https://youtu.be/yWqX8wo5alQ , title: [ Woman Left Child, Husband, Family Because of U.G. She Says. Hear His Reply... ]
     "I'm not responsible."  Books on U.G. Krishnamurti: Mind is a Myth - https://amz

https://youtu.be/WrpQcFJOFQ4 , title: [ U.G. Krishnamurti on Parveen Babi ]
     January 1990.  Books on U.G. Krishnamurti: Mind is a Myth - https://amzn.to/3erG

https://youtu.be/kKkBzHrWSVY , title: [ U.G. Krishnamurti - You Can Never Be Permanently Happy, Nor Have Permanent Pleasure ]
     After a date check U.G. blasts in and on. “Never trust a man who says he is conc

https://youtu.be/1elfvTKqdVk , title: [ U.G. Krishnamurti - You Have Lost Your Properties… ]
     Short clip, hearable but not good audio. Just for the record.  Books on U.G. Kri

https://youtu.be/aP-JiJxcz3o , title: [ U.G. Krishnamurti – What Is the Mind You Are Talking Of? ]
     U.G. mentions how he walked and talked with cobra snakes…  Books on U.G. Krishna

https://youtu.be/r_b4LoxR0Mk , title: [ U.G. Krishn

In [69]:
# Search for videos from channel by channel ID
host = "www.googleapis.com"
path = "youtube/v3/search"
query = {
    "key": GOOGLE_API_KEY,
    "channelId": "UCRLQIvQcoF41xZdrtKMXSvg",
    "maxResults": 20,
    "part": "snippet",
    "regionCode": "CA",
    "safeSearch": "none",
    "type": "video"
}
query = query_to_str(query)
query = f"https://{host}/{path}?{query}"
try:
    res = requests.get(query)
    print("Status code:", res.status_code)
    payload = json.loads(res.content)
except Exception as e:
    print("Exception:", e)

# each search result (in items) is one of
# https://developers.google.com/resources/api-libraries/documentation/youtube/v3/csharp/latest/classGoogle_1_1Apis_1_1YouTube_1_1v3_1_1Data_1_1SearchResult.html
print()
print("===Content===")
print("pageInfo:")
for k, v in payload["pageInfo"].items():
    print("    ", k, v)
print("regionCode", payload["regionCode"])
print("prevPageToken:", payload.get("prevPageToken", None))
print("nextPageToken:", payload.get("nextPageToken", None))

print()
print("===Item 0===")
item = payload["items"][0]
videoId = item["id"]["videoId"]
title = snippet["title"]
description = snippet["description"]
print("title:", title)
print("videoId:", videoId)
print("description:", description.replace("\n"," ")[:200])
# each snippet is one of
snippet = item["snippet"]
print(snippet["publishedAt"])
# Date and time of publication in format:
# https://www.w3.org/TR/NOTE-datetime
# strptime() explained in:
# https://docs.python.org/3/library/time.html#time.strftime
publishedAt = datetime.strptime(snippet["publishedAt"], "%Y-%m-%dT%H:%M:%SZ")
print("publishedAt:", publishedAt)

channelTitle = snippet["channelTitle"]
channelId = snippet["channelId"]
print("From channel", channelTitle, ", channelId:", channelId)

Status code: 200

===Content===
pageInfo:
     totalResults 320
     resultsPerPage 20
regionCode CA
prevPageToken: None
nextPageToken: CBQQAA

===Item 0===
title: U.G. Krishnamurti - You&#39;re Doing Nothing!
videoId: gtXvDttcKgg
description: Short and to the point clip of U.G. slicing up the man's ideas. One has got to love the word 'nothing' after all watching some videos ...
2015-12-23T07:42:54Z
publishedAt: 2015-12-23 07:42:54
From channel Lectures Beyond Beyond , channelId: UCRLQIvQcoF41xZdrtKMXSvg


In [89]:
# Get all the videos from a channel using search.list, given channel ID
pageToken = None
count = 0
items = []
while True:
    host = "www.googleapis.com"
    path = "youtube/v3/search"
    query = {
        "key": GOOGLE_API_KEY,
        "channelId": "UCRLQIvQcoF41xZdrtKMXSvg",
        "maxResults": 50,
        "part": "snippet",
        "regionCode": "CA",
        "safeSearch": "none",
        "type": "video",
        "pageToken": pageToken
    }
    query = query_to_str(query)
    query = f"https://{host}/{path}?{query}"
    try:
        res = requests.get(query)
        payload = json.loads(res.content)
        items.extend(payload["items"])
        count += 1
        print("o ", end=" ")
        try:
            pageToken = payload["nextPageToken"]
        except KeyError:
            break    
    except Exception as e:
        print("Exception:", e)

print()
print(f"Made {count} requests")
print(f"Got {len(items)} videos")

o  o  o  o  o  o  o  
Made 7 requests
Got 309 videos


In [90]:
# List all the videos we go earlier, and print the links to the videos
should_print_link = True
for item in items:
    videoId = item["id"]["videoId"]
    snippet = item["snippet"]
    title = snippet["title"]
    description = snippet["description"]
    description = description.replace("\n"," ")[:80]
    if should_print_link:
        print(f"https://youtu.be/{videoId}", end=" ")
        print(f", title: [ {title} ]")
    else:
        print(f"videoId: {videoId} , title: [ {title} ]")
    print("    ", description)
    print()

https://youtu.be/gtXvDttcKgg , title: [ U.G. Krishnamurti - You&#39;re Doing Nothing! ]
     Short and to the point clip of U.G. slicing up the man's ideas. One has got to l

https://youtu.be/OUPWLRrkN1Y , title: [ U.G. Krishnamurti - Your Expectation, Your Delusion ]
     Books on U.G. Krishnamurti: Mind is a Myth - https://amzn.to/3erGeYi The Courage

https://youtu.be/6deGa7wkidw , title: [ U.G. Krishnamurti - Thoughts, Language &amp; Energy ]
     Books on U.G. Krishnamurti: Mind is a Myth - https://amzn.to/3erGeYi The Courage

https://youtu.be/knm0vDweuOc , title: [ U.G. Krishnamurti - Negative Philosophy ]
     Interview excerpt with Terry Newland. "I don't trust a man who wants to be selfl

https://youtu.be/VxvPQwhzKpE , title: [ U.G. Krishnamurti - Material &amp; Spiritual ]
     Books on U.G. Krishnamurti: Mind is a Myth - https://amzn.to/3erGeYi The Courage

https://youtu.be/TiMFeBce_2Y , title: [ U.G. Krishnamurti - Energy Levels ]
     Books on U.G. Krishnamurti: Mind is a M

In [79]:
# Get subscription data from a user/channel
# possible to get 403, since subscriptions are private
# need to use InstalledAppFlow
host = "www.googleapis.com"
path = "youtube/v3/subscriptions"
part = "channelId=UCSk6p_put_fIDe7vXdw_PDQ"
query = f"https://{host}/{path}?{part}&key={GOOGLE_API_KEY}"
try:
    res = requests.get(query)
    print("Status code:", res.status_code)
    payload = json.loads(res.content)
except Exception as e:
    print("Exception:", e)
payload

Status code: 403


{'error': {'code': 403,
  'message': 'The requester is not allowed to access the requested subscriptions.',
  'errors': [{'message': 'The requester is not allowed to access the requested subscriptions.',
    'domain': 'youtube.subscription',
    'reason': 'subscriptionForbidden'}]}}

## OAuth 2.0

In [2]:
# Get OAuth 2.0 authenticated session to get my YT private data.

# Based on:
# https://github.com/youtube/api-samples

# The CLIENT_SECRETS_FILE specifies the name of a file that contains
# OAuth 2.0 client ID and client secret from Google Cloud Console at
# https://cloud.google.com/console

# Need to enable the YouTube Data API for your project.
# See: https://developers.google.com/youtube/v3/guides/authentication
CLIENT_SECRETS_FILE = '../.secrets/client_secret.json'

# This OAuth 2.0 access scope allows for full read/write access to the
# authenticated user's account.
SCOPES = ['https://www.googleapis.com/auth/youtube']
API_SERVICE_NAME = 'youtube'
API_VERSION = 'v3'

flow = InstalledAppFlow.from_client_secrets_file(CLIENT_SECRETS_FILE, SCOPES)
credentials = flow.run_console()
youtube = build(API_SERVICE_NAME, API_VERSION, credentials=credentials)

Please visit this URL to authorize this application: https://accounts.google.com/o/oauth2/auth?response_type=code&client_id=517970623738-ctnfm8dhn55jbmrtsn1kmb4i68isl2ah.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fyoutube&state=4nJWLDWfSokA5UDaaC1QdC0Pe4nA73&prompt=consent&access_type=offline
Enter the authorization code: 4/1AX4XfWgol6Xnpf2oe2COjSeXImyjF9mEer6BtpdY3TEYsSXItHzx4KmYekQ


In [200]:
# Use session to query for my list of subscribers
# See: https://developers.google.com/youtube/v3/docs/subscriptions/list
payload = youtube.subscriptions().list(
    part="id,contentDetails,snippet,subscriberSnippet",
    maxResults=50,
    mine=True,
#     pageToken="CDIQAA"
).execute()

In [201]:
# Print the results of subscriptions.list
# See also:
# https://developers.google.com/resources/api-libraries/documentation/youtube/v3/csharp/latest/classGoogle_1_1Apis_1_1YouTube_1_1v3_1_1Data_1_1SubscriptionContentDetails.html
print("pageInfo:", payload["pageInfo"])
print("prevPageToken:", payload.get("prevPageToken", None))
print("nextPageToken:", payload.get("nextPageToken", None))

item = payload["items"][0]
subscriptionId = item["id"]
snippet = item["snippet"]
contentDetails = item["contentDetails"]
subscriberSnippet = item["subscriberSnippet"]

print()
print("contentDetails:")
# ActivityType : The type of activity this subscription is for (only uploads, everything).
# NewItemCount : The number of new items in the subscription since its content was last read.
# TotalItemCount : The approximate number of items that the subscription points to.
pp.pprint(contentDetails)
print("Subscription ID:", subscriptionId)
print("channelId:", snippet["resourceId"]["channelId"])
print("title:", snippet["title"])
print("description:")
print("[[[", description.replace("\n", " ")[:200], "]]]")
# pp.pprint(snippet)
# pp.pprint(subscriberSnippet)

pageInfo: {'totalResults': 530, 'resultsPerPage': 50}
prevPageToken: None
nextPageToken: CDIQAA

contentDetails:
{'activityType': 'all', 'newItemCount': 0, 'totalItemCount': 353}
Subscription ID: Hi7kfV3Wm_C0r2SC_Pso2898maAsQbjHU9z0PHvoDww
channelId: UC1pdbN6j_txoNhXVWVtouuQ
title: World Armwrestling League
description:
[[[ Part 1: The Moon and Stars ━━━━━━━━━━━━━━━━━━━━━━━━━━ Follow me on Twitter! https://twitter.com/realMax0r Patreon: https://www.patreon.com/max0r Join the Max0r Discord! https://discord.gg/max0r Twitch ]]]


Made 11 requests
Got 531 subscriptions


channelId: UCsXV4t9YfUeDuZ276hE20Jg , title: [ Adiyogi ], 346 videos
channelId: UC1pdbN6j_txoNhXVWVtouuQ , title: [ World Armwrestling League ], 353 videos
channelId: UCGGZ_POGmIWG1pQXTDzQv-g , title: [ Michelle Khare ], 189 videos
channelId: UCeUJFQ0D9qs6aVNyUt9fkeQ , title: [ The Armchair Historian ], 138 videos
channelId: UCMbvTF91O--bhwmg9dTMAPA , title: [ Lyle Forever ], 242 videos
channelId: UCrDQW9kAElm707c5z6d5r7Q , title: [ albie ], 62 videos
channelId: UCfgh3Ul_dG6plQ7rzuOLx-w , title: [ Max0r ], 81 videos
channelId: UC6JhS4GvWf3AJfOTfkrse2w , title: [ Simulation ], 1490 videos
channelId: UCvQ9wJKXJwcw_4tNaHHbWdA , title: [ Speak The Truth ], 291 videos
channelId: UCoydhtfFSk1fZXNRnkGnneQ , title: [ ESOTERICA ], 167 videos
channelId: UCE9htqwsu0II6KrpVKdrXHw , title: [ Zuck That ], 18 videos
channelId: UCHFYJPHg6mO0uEMhIla8lFQ , title: [ Content Savant ], 126 videos
channelId: UCkySD00cmDWYHXA31hqRYRw , title: [ Gennady Korotkevich ], 20 videos
channelId: UCOKfTeI3E2VVOqqV_KP